In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('topic_modeling.csv')
#data = pd.read_csv('all.csv')

In [3]:
#data = data[['PY','PT','AF','PU','FU','PA','PG']]
data.head()

,Cluster_Topic,PY,Topic_Year,PT,AF,PU,FU,PA,PG
0,0,2000,0_2000,J,"Jandt, KD; Finke, M; Cacciafesta, P",ELSEVIER SCIENCE BV,NaN,"PO BOX 211, 1000 AE AMSTERDAM, NETHERLANDS",14
1,7,2000,7_2000,J,"Burger, J; Hummel, S; Herrmann, B",ELSEVIER SCIENCE BV,NaN,"PO BOX 211, 1000 AE AMSTERDAM, NETHERLANDS",6
2,11,2000,11_2000,J,"Kaibara, M; Takahashi, A; Kurotobi, K; Suzuki, Y",ELSEVIER SCIENCE BV,NaN,"PO BOX 211, 1000 AE AMSTERDAM, NETHERLANDS",9
3,14,2000,14_2000,J,"Mano, JF; Reis, RL; Cunha, AM",JOHN WILEY & SONS INC,NaN,"605 THIRD AVE, NEW YORK, NY 10158-0012 USA",13
4,5,2000,5_2000,J,"Chrisey, DB; Pique, A; Modi, R; Wu, HD; Auyeun...",ELSEVIER SCIENCE BV,NaN,"PO BOX 211, 1000 AE AMSTERDAM, NETHERLANDS",8


In [4]:
# Need to address missing data issue.
data.isnull().sum()

Cluster_Topic        0
PY                   0
Topic_Year           0
PT                   0
AF                   2
PU                   1
FU               20173
PA                   1
PG                   0
dtype: int64

In [5]:
# PY contains two missing records.
# Drop off these two records.
#data = data.dropna(axis=0,subset=['PY'])

# Top authors in each year
- For each year, extract the top authors who contributed the most in the top list in terms of the number of publications in previous years.
- For instance, to find the top authors in 2013, we should consider top authors who published before 2013 rather than only 2013.

In [7]:
# Impute missing values in 'AF' column.
data['AF'].fillna('NA author', inplace=True)

In [8]:
data['All_authors'] = data.AF.str.split(';')

In [9]:
data['All_authors'].head()

0              [Jandt, KD,  Finke, M,  Cacciafesta, P]
1                [Burger, J,  Hummel, S,  Herrmann, B]
2    [Kaibara, M,  Takahashi, A,  Kurotobi, K,  Suz...
3                    [Mano, JF,  Reis, RL,  Cunha, AM]
4    [Chrisey, DB,  Pique, A,  Modi, R,  Wu, HD,  A...
Name: All_authors, dtype: object

In [10]:
# Make sure no missing value in 'AF' and 'All_authors'.
data.isnull().sum()

Cluster_Topic        0
PY                   0
Topic_Year           0
PT                   0
AF                   0
PU                   1
FU               20173
PA                   1
PG                   0
All_authors          0
dtype: int64

In [11]:
def top_author_in_year(year,index,dataframe):
    '''
    year: a target year.
    index: an index number to determine how many top authors should be extracted.
    dataframe: data stored in a pandas frame.
    Output: Return a list of top authors before a target year given an index number.
    '''
    
    df = dataframe
    grouped_df_in_year = df[df['PY'] <= year]
    author_in_year = grouped_df_in_year['All_authors']
    author_count_dic = {}
    for authors in author_in_year:
        for author in authors:
            if author not in author_count_dic:
                author_count_dic[author.strip().replace('.','')] = 1
            else: 
                author_count_dic[author.strip().replace('.','')] += 1
    sorted_author_count_dic = sorted(author_count_dic.items(), key=lambda kv: kv[1])[::-1]
    top_authors = []
    for i in range(index):
        top_authors.append(sorted_author_count_dic[i][0])
        
    # Consider 'tied' authors.
    for each in sorted_author_count_dic[index:]:
        if each[1] == sorted_author_count_dic[index-1][1]:
            top_authors.append(each[0])
        
    return top_authors

In [12]:
# Collect top authors in each year.
year = []
top_authors = []
for i in range(1973, 2020):
    year.append(i)
    top_authors.append(top_author_in_year(year=i-1,index=10,dataframe=data))

In [13]:
top_author_df = pd.DataFrame({'PY': year, 'top_authors_in_previous_yrs': top_authors})
top_author_df.head()

,PY,top_authors_in_previous_yrs
0,1973,"[SALZMAN, EW, MERRILL, EW, BRUCK, SD, VANHOEK,..."
1,1974,"[SALZMAN, EW, MERRILL, EW, BRUCK, SD, VANHOEK,..."
2,1975,"[SALZMAN, EW, MERRILL, EW, MASON, RG, ANDERSON..."
3,1976,"[MASON, RG, HULBERT, SF, SALZMAN, EW, MERRILL,..."
4,1977,"[MASON, RG, LEMONS, JE, WILLIAMS, DF, HULBERT,..."


In [14]:
# Make sure collect all top authors from every year.
top_author_df.PY.describe()[['min','max']]

min    1973.0
max    2019.0
Name: PY, dtype: float64

# Top publishers in each year
- For each year, extract top publishers which published the most in the top list in terms of the number of publications in previous years.

In [15]:
# Impute missing values in 'PU' column.
data['PU'].fillna('NA publisher', inplace=True)

In [16]:
# Make sure no missing data in 'PU'.
data['PU'].isnull().sum()

0

In [17]:
def top_publisher_in_year(year,index,dataframe):
    '''
    year: a target year.
    index: an index number to determine how many top publishers should be extracted.
    dataframe: data stored in a pandas frame.
    Output: Return a list of top publishers before a target year given an index number.
    '''
    
    df = dataframe
    grouped_df_in_year = df[df['PY'] <= year]
    publisher_in_year = grouped_df_in_year['PU']
    publisher_count_dic = {}
    for publisher in publisher_in_year:
        if publisher not in publisher_count_dic:
            publisher_count_dic[publisher] = 1
        else: 
            publisher_count_dic[publisher] += 1
    
    sorted_publisher_count_dic = sorted(publisher_count_dic.items(), key=lambda kv: kv[1])[::-1]
    top_publishers = []
    
    if len(sorted_publisher_count_dic) < index:
        for each in sorted_publisher_count_dic:
            top_publishers.append(each[0])
    else:    
        for i in range(index):
            top_publishers.append(sorted_publisher_count_dic[i][0])
        
    # Consider 'tied' publishers.
    for each in sorted_publisher_count_dic[index:]:
        if each[1] == sorted_publisher_count_dic[index-1][1]:
            top_publishers.append(each[0])
        
    return top_publishers

In [18]:
# Collect top publishers in each year.
year = []
top_publishers = []
for i in range(1973, 2020):
    year.append(i)
    top_publishers.append(top_publisher_in_year(year=i-1,index=10,dataframe=data))

In [19]:
top_publisher_df = pd.DataFrame({'PY': year, 'top_publishers_in_previous_yrs': top_publishers})

In [20]:
top_publisher_df.head()

,PY,top_publishers_in_previous_yrs
0,1973,"[NEW YORK ACAD MEDICINE, AMER CHEMICAL SOC, GE..."
1,1974,"[NEW YORK ACAD MEDICINE, AMER CHEMICAL SOC, JO..."
2,1975,"[NEW YORK ACAD MEDICINE, JOHN WILEY & SONS INC..."
3,1976,"[JOHN WILEY & SONS INC, NEW YORK ACAD MEDICINE..."
4,1977,"[JOHN WILEY & SONS INC, NEW YORK ACAD MEDICINE..."


In [21]:
# Make sure collect all top publishers from every year.
top_publisher_df.PY.describe()[['min','max']]

min    1973.0
max    2019.0
Name: PY, dtype: float64

# Top funding agencies in each year
- For each year, extract the top funding agencies which funded the most in the top list in terms of the number of publications in previous years.

In [22]:
# Impute missing values in 'AF' column.
data['FU'].fillna('NA funding agency', inplace=True)

In [23]:
data['All_funding_agencies'] = data.FU.str.split(';')

In [24]:
data['All_funding_agencies'].head()

0    [NA funding agency]
1    [NA funding agency]
2    [NA funding agency]
3    [NA funding agency]
4    [NA funding agency]
Name: All_funding_agencies, dtype: object

In [25]:
# Make sure no missing data in 'FU' and 'All_funding_agencies'.
data.isnull().sum()

Cluster_Topic           0
PY                      0
Topic_Year              0
PT                      0
AF                      0
PU                      0
FU                      0
PA                      1
PG                      0
All_authors             0
All_funding_agencies    0
dtype: int64

In [26]:
def top_funding_agency_in_year(year,index,dataframe):
    '''
    year: a target year.
    index: an index number to determine how many top funding agencies should be extracted.
    dataframe: data stored in a pandas frame.
    Output: Return a list of top funding agencies before a target year given an index number.
    '''
    
    df = dataframe
    grouped_df_in_year = df[df['PY'] <= year]
    funding_agency_in_year = grouped_df_in_year['All_funding_agencies']
    funding_agency_count_dic = {}
    
    for funding_agencies in funding_agency_in_year:
        for funding_agency in funding_agencies:
            if funding_agency not in funding_agency_count_dic:
                funding_agency_count_dic[funding_agency] = 1
            else: 
                funding_agency_count_dic[funding_agency] += 1
                
    sorted_funding_agency_count_dic = sorted(funding_agency_count_dic.items(), key=lambda kv: kv[1])[::-1][1:]
    top_funding_agencies = []
    
    if len(sorted_funding_agency_count_dic) < index:
        for each in sorted_funding_agency_count_dic:
            top_funding_agencies.append(each[0])
    else:    
        for i in range(index):
            top_funding_agencies.append(sorted_funding_agency_count_dic[i][0])
        
    # Consider 'tied' funding agencies.
    for each in sorted_funding_agency_count_dic[index:]:
        if each[1] == sorted_funding_agency_count_dic[index-1][1]:
            top_funding_agencies.append(each[0])
        
    return top_funding_agencies

In [27]:
# Collect top funding agencies in each year.
year = []
top_funding_agencies = []
for i in range(1973, 2020):
    year.append(i)
    top_funding_agencies.append(top_funding_agency_in_year(year=i-1,index=10,dataframe=data))

In [28]:
top_funding_agency_df = pd.DataFrame({'PY': year, 'top_funding_agency_in_previous_yrs': top_funding_agencies})

In [29]:
# From 1973 to 1980, there were no funding agencies.
top_funding_agency_df.head(10)

,PY,top_funding_agency_in_previous_yrs
0,1973,[]
1,1974,[]
2,1975,[]
3,1976,[]
4,1977,[]
5,1978,[]
6,1979,[]
7,1980,[]
8,1981,[NIGMS NIH HHS [GM 24380]]
9,1982,"[NIGMS NIH HHS [GM 24380], NHLBI NIH HHS [N01-..."


# Have funding agency
- Because originally, there are many missing data in 'FU'. Create a boolean type of variable to indicate whether there is a funding agency or not.

In [30]:
def have_funding_agency(x):
    if x == 'NA funding agency':
        return 0
    else:
        if type(x) == list:
            return len(x)
        else:
            return 1

In [31]:
data['Have_Funding_Agency'] = data['FU'].apply(have_funding_agency)

In [32]:
data[['FU','Have_Funding_Agency']].head(10)

,FU,Have_Funding_Agency
0,NA funding agency,0
1,NA funding agency,0
2,NA funding agency,0
3,NA funding agency,0
4,NA funding agency,0
5,NA funding agency,0
6,"NIDCR NIH HHS [DE12350, DE02848]",1
7,NA funding agency,0
8,NA funding agency,0
9,NA funding agency,0


# Count of author
- Publications might have several authors so perhaps more authors could bring more attentation on their publications.

In [33]:
data['Author_count'] = data['All_authors'].apply(len)

In [34]:
data[['All_authors','Author_count']].head(10)

,All_authors,Author_count
0,"[Jandt, KD, Finke, M, Cacciafesta, P]",3
1,"[Burger, J, Hummel, S, Herrmann, B]",3
2,"[Kaibara, M, Takahashi, A, Kurotobi, K, Suz...",4
3,"[Mano, JF, Reis, RL, Cunha, AM]",3
4,"[Chrisey, DB, Pique, A, Modi, R, Wu, HD, A...",6
5,"[Nkamgueu, EM, Adnet, JJ, Bernard, J, Ziero...",8
6,"[Wen, HB, Moradian-Oldak, J, Zhong, JP, Gre...",5
7,"[Ding, YM]",1
8,"[Hirsch, R, Katz, E, Willner, I]",3
9,"[Chapekar, MS]",1


# Count of funding agency
- Number of funding agency. Probably, more funding agencies to sponsor a publication could bring more attentation on it.

In [35]:
def count_funding_agency(x):
    if x == ['NA funding agency']:
        return 0
    else:
        return len(x)

In [36]:
data['Funding_agency_count'] = data['All_funding_agencies'].apply(count_funding_agency)

In [37]:
data[['Funding_agency_count', 'All_funding_agencies']].head(10)

,Funding_agency_count,All_funding_agencies
0,0,[NA funding agency]
1,0,[NA funding agency]
2,0,[NA funding agency]
3,0,[NA funding agency]
4,0,[NA funding agency]
5,0,[NA funding agency]
6,1,"[NIDCR NIH HHS [DE12350, DE02848]]"
7,0,[NA funding agency]
8,0,[NA funding agency]
9,0,[NA funding agency]


# Publisher
- Lower down the levels in Publisher into seven levels by only considering top publishers in recent years from 2008 to 2018.

In [38]:
def squeeze_publisher(x):
    top_publishers = ['ELSEVIER SCI LTD', 'ELSEVIER SCIENCE BV', 'AMER CHEMICAL SOC', 'WILEY',\
                      'SPRINGER', 'WILEY-BLACKWELL', 'ROYAL SOC CHEMISTRY', 'WILEY-V C H VERLAG GMBH',\
                      'ELSEVIER SCIENCE SA', 'PERGAMON-ELSEVIER SCIENCE LTD']
    if x.strip().upper() in top_publishers:
        return x.strip().upper()
    else:
        return 'Other Publishers'

In [39]:
data['Squeezed_Publisher'] = data.PU.apply(squeeze_publisher)

In [40]:
data[['PU','Squeezed_Publisher']].head(10)

,PU,Squeezed_Publisher
0,ELSEVIER SCIENCE BV,ELSEVIER SCIENCE BV
1,ELSEVIER SCIENCE BV,ELSEVIER SCIENCE BV
2,ELSEVIER SCIENCE BV,ELSEVIER SCIENCE BV
3,JOHN WILEY & SONS INC,Other Publishers
4,ELSEVIER SCIENCE BV,ELSEVIER SCIENCE BV
5,JOHN WILEY & SONS INC,Other Publishers
6,JOHN WILEY & SONS INC,Other Publishers
7,AMER ASSOC ADVANCEMENT SCIENCE,Other Publishers
8,AMER CHEMICAL SOC,AMER CHEMICAL SOC
9,JOHN WILEY & SONS INC,Other Publishers


In [41]:
# Make sure the transformation is correct!
# Must be 11 levels.
len(data.Squeezed_Publisher.unique())

11

# Country (Skipped this part)

In [ ]:
# Impute missing values in 'PA' column.
data['PA'].fillna('NA country', inplace=True)

In [ ]:
def extract_country(x):
    target_country = ('usa','china','germany','japan','england','italy','korea',\
                      'france','india','canada','spain','australia','netherlands','portugal',\
                      'switzerland','sweden')
    split_country = x.strip().lower().split(' ')
    if any(each in target_country for each in split_country):
        if split_country[-1] in target_country:
            return split_country[-1].upper()
        elif split_country[-2] in target_country:
            return split_country[-2].upper()
        elif split_country[-3] in target_country:
            return split_country[-3].upper()
    else:
        return 'Other countries'
        
        #search_country = x.split(' ')
        #total_len = len(search_country)
        #for i in range(total_len):
            #if search_country[i] in target_country:
                #final_country = search_country[i]
                
                #return final_country

# Final Data

In [42]:
# First merge the original dataframe with new feature each by each on the key 'PY' (or year).
print(top_author_df.head())
print(top_publisher_df.head())
print(top_funding_agency_df.head())

     PY                        top_authors_in_previous_yrs
0  1973  [SALZMAN, EW, MERRILL, EW, BRUCK, SD, VANHOEK,...
1  1974  [SALZMAN, EW, MERRILL, EW, BRUCK, SD, VANHOEK,...
2  1975  [SALZMAN, EW, MERRILL, EW, MASON, RG, ANDERSON...
3  1976  [MASON, RG, HULBERT, SF, SALZMAN, EW, MERRILL,...
4  1977  [MASON, RG, LEMONS, JE, WILLIAMS, DF, HULBERT,...
     PY                     top_publishers_in_previous_yrs
0  1973  [NEW YORK ACAD MEDICINE, AMER CHEMICAL SOC, GE...
1  1974  [NEW YORK ACAD MEDICINE, AMER CHEMICAL SOC, JO...
2  1975  [NEW YORK ACAD MEDICINE, JOHN WILEY & SONS INC...
3  1976  [JOHN WILEY & SONS INC, NEW YORK ACAD MEDICINE...
4  1977  [JOHN WILEY & SONS INC, NEW YORK ACAD MEDICINE...
     PY top_funding_agency_in_previous_yrs
0  1973                                 []
1  1974                                 []
2  1975                                 []
3  1976                                 []
4  1977                                 []


In [43]:
# Shape before merging.
data.shape

(46169, 15)

In [44]:
new_data = pd.merge(data, top_author_df, on='PY', how='left')
new_data = pd.merge(new_data, top_publisher_df, on='PY', how='left')
new_data = pd.merge(new_data, top_funding_agency_df, on='PY', how='left')
new_data.shape

(46169, 18)

In [45]:
new_data.head()

,Cluster_Topic,PY,Topic_Year,PT,AF,PU,FU,PA,PG,All_authors,All_funding_agencies,Have_Funding_Agency,Author_count,Funding_agency_count,Squeezed_Publisher,top_authors_in_previous_yrs,top_publishers_in_previous_yrs,top_funding_agency_in_previous_yrs
0,0,2000,0_2000,J,"Jandt, KD; Finke, M; Cacciafesta, P",ELSEVIER SCIENCE BV,NA funding agency,"PO BOX 211, 1000 AE AMSTERDAM, NETHERLANDS",14,"[Jandt, KD, Finke, M, Cacciafesta, P]",[NA funding agency],0,3,0,ELSEVIER SCIENCE BV,"[BRUCK, SD, [Anonymous], HEIMKE, G, DANIELS, A...","[JOHN WILEY & SONS INC, ELSEVIER SCI LTD, AMER...","[Wellcome Trust, NHLBI NIH HHS [HL 39081], NCR..."
1,7,2000,7_2000,J,"Burger, J; Hummel, S; Herrmann, B",ELSEVIER SCIENCE BV,NA funding agency,"PO BOX 211, 1000 AE AMSTERDAM, NETHERLANDS",6,"[Burger, J, Hummel, S, Herrmann, B]",[NA funding agency],0,3,0,ELSEVIER SCIENCE BV,"[BRUCK, SD, [Anonymous], HEIMKE, G, DANIELS, A...","[JOHN WILEY & SONS INC, ELSEVIER SCI LTD, AMER...","[Wellcome Trust, NHLBI NIH HHS [HL 39081], NCR..."
2,11,2000,11_2000,J,"Kaibara, M; Takahashi, A; Kurotobi, K; Suzuki, Y",ELSEVIER SCIENCE BV,NA funding agency,"PO BOX 211, 1000 AE AMSTERDAM, NETHERLANDS",9,"[Kaibara, M, Takahashi, A, Kurotobi, K, Suz...",[NA funding agency],0,4,0,ELSEVIER SCIENCE BV,"[BRUCK, SD, [Anonymous], HEIMKE, G, DANIELS, A...","[JOHN WILEY & SONS INC, ELSEVIER SCI LTD, AMER...","[Wellcome Trust, NHLBI NIH HHS [HL 39081], NCR..."
3,14,2000,14_2000,J,"Mano, JF; Reis, RL; Cunha, AM",JOHN WILEY & SONS INC,NA funding agency,"605 THIRD AVE, NEW YORK, NY 10158-0012 USA",13,"[Mano, JF, Reis, RL, Cunha, AM]",[NA funding agency],0,3,0,Other Publishers,"[BRUCK, SD, [Anonymous], HEIMKE, G, DANIELS, A...","[JOHN WILEY & SONS INC, ELSEVIER SCI LTD, AMER...","[Wellcome Trust, NHLBI NIH HHS [HL 39081], NCR..."
4,5,2000,5_2000,J,"Chrisey, DB; Pique, A; Modi, R; Wu, HD; Auyeun...",ELSEVIER SCIENCE BV,NA funding agency,"PO BOX 211, 1000 AE AMSTERDAM, NETHERLANDS",8,"[Chrisey, DB, Pique, A, Modi, R, Wu, HD, A...",[NA funding agency],0,6,0,ELSEVIER SCIENCE BV,"[BRUCK, SD, [Anonymous], HEIMKE, G, DANIELS, A...","[JOHN WILEY & SONS INC, ELSEVIER SCI LTD, AMER...","[Wellcome Trust, NHLBI NIH HHS [HL 39081], NCR..."


In [46]:
# Because 14 records are in 1972 (oldest year), thus 14 missing values in top-related features.
new_data.isnull().sum()

Cluster_Topic                          0
PY                                     0
Topic_Year                             0
PT                                     0
AF                                     0
PU                                     0
FU                                     0
PA                                     1
PG                                     0
All_authors                            0
All_funding_agencies                   0
Have_Funding_Agency                    0
Author_count                           0
Funding_agency_count                   0
Squeezed_Publisher                     0
top_authors_in_previous_yrs           14
top_publishers_in_previous_yrs        14
top_funding_agency_in_previous_yrs    14
dtype: int64

In [47]:
# Boolean feature
def contain_top_feature(feature_name,top_feature_name,dataframe):
    #total_len = dataframe[dataframe.PY>1999].shape[0]
    #filtered_df = dataframe[dataframe.PY>1999]
    #filtered_df = filtered_df.reset_index()
    
    total_len = dataframe.shape[0]
    filtered_df = dataframe
    filtered_df = filtered_df.reset_index()
    
    index_contain_top_feature = []
    boolean_value = []
    for i in range(total_len):
        total_features = filtered_df[feature_name][i]
        search_features = filtered_df[top_feature_name][i]
        if type(search_features) == float:
            index_contain_top_feature.append(i)
            boolean_value.append(0)
        else:
            for feat in total_features:
                if feat in search_features:
                    index_contain_top_feature.append(i)
                    boolean_value.append(1)
                    break
                else:
                    continue
    contain_top_feature_df = pd.DataFrame({'Contain_Top_'+feature_name:boolean_value},\
                                          index=index_contain_top_feature)
    return contain_top_feature_df

In [48]:
# Count multiple features
def contain_multiple_top_feature(feature_name,top_feature_name,dataframe):
    #total_len = dataframe[dataframe.PY>1999].shape[0]
    #filtered_df = dataframe[dataframe.PY>1999]
    #filtered_df = filtered_df.reset_index()
    
    total_len = dataframe.shape[0]
    filtered_df = dataframe
    filtered_df = filtered_df.reset_index()
    
    count_top_feature = []
    for i in range(total_len):
        total_features = filtered_df[feature_name][i]
        search_features = filtered_df[top_feature_name][i]
        if type(search_features) == float:
            count_top_feature.append(0)
        else:
            total_sum = 0
            for feat in total_features:
                if feat in search_features:
                    total_sum += 1
            count_top_feature.append(total_sum)
    
    return count_top_feature

In [49]:
# Top authors & funding agencies.
contain_top_author_df = contain_top_feature(feature_name='All_authors',
                                            top_feature_name='top_authors_in_previous_yrs',
                                            dataframe=new_data)
contain_top_fund_agency_df = contain_top_feature(feature_name='All_funding_agencies',
                                            top_feature_name='top_funding_agency_in_previous_yrs',
                                            dataframe=new_data)

In [50]:
#target_df = new_data[new_data.PY>2000]
target_df = new_data.reset_index()
target_df = target_df.merge(contain_top_author_df, how='left', left_index=True, right_index=True)
target_df = target_df.merge(contain_top_fund_agency_df, how='left', left_index=True, right_index=True)

In [51]:
target_df.drop('index',axis=1,inplace=True)

In [52]:
# Count of multiple funding agencies. 
# (For multiple top authors, there is always one top author. Thus, using only contain_top_author_df.
count_mult_funding_agency_ls = contain_multiple_top_feature(feature_name='All_funding_agencies',
                                                        top_feature_name='top_funding_agency_in_previous_yrs',
                                                        dataframe=target_df)

In [53]:
target_df['Count_of_top_funding_agency'] = count_mult_funding_agency_ls

In [56]:
def contain_top_feature(feature_name,top_feature_name,dataframe):
    #total_len = dataframe[dataframe.PY>2000].shape[0]
    #filtered_df = dataframe[dataframe.PY>2000]
    #filtered_df = filtered_df.reset_index()
    
        
    total_len = dataframe.shape[0]
    filtered_df = dataframe
    filtered_df = filtered_df.reset_index()


    index_contain_top_feature = []
    boolean_value = []
    for i in range(total_len):
        total_features = filtered_df[feature_name][i]
        search_features = filtered_df[top_feature_name][i]
        if type(search_features) == float:
            index_contain_top_feature.append(i)
            boolean_value.append(0)
        elif total_features in search_features:
            index_contain_top_feature.append(i)
            boolean_value.append(1)
            
    contain_top_feature_df = pd.DataFrame({'Contain_Top_'+feature_name:boolean_value},\
                                          index=index_contain_top_feature)
    return contain_top_feature_df

In [57]:
contain_top_publisher_df = contain_top_feature(feature_name='PU',
                                               top_feature_name='top_publishers_in_previous_yrs',
                                               dataframe=new_data)

In [58]:
target_df = target_df.merge(contain_top_publisher_df, how='left', left_index=True, right_index=True)

In [59]:
target_df['Contain_Top_All_authors'].fillna(0, inplace=True)
target_df['Contain_Top_All_funding_agencies'].fillna(0, inplace=True)
target_df['Contain_Top_PU'].fillna(0, inplace=True)

In [60]:
target_df.isnull().sum()

Cluster_Topic                          0
PY                                     0
Topic_Year                             0
PT                                     0
AF                                     0
PU                                     0
FU                                     0
PA                                     1
PG                                     0
All_authors                            0
All_funding_agencies                   0
Have_Funding_Agency                    0
Author_count                           0
Funding_agency_count                   0
Squeezed_Publisher                     0
top_authors_in_previous_yrs           14
top_publishers_in_previous_yrs        14
top_funding_agency_in_previous_yrs    14
Contain_Top_All_authors                0
Contain_Top_All_funding_agencies       0
Count_of_top_funding_agency            0
Contain_Top_PU                         0
dtype: int64

In [61]:
# Drop off columns
drop_cols = ['AF','PU','FU','PA','All_authors','All_funding_agencies',\
             'top_authors_in_previous_yrs','top_publishers_in_previous_yrs','top_funding_agency_in_previous_yrs']
target_df.drop(drop_cols, axis=1,inplace=True)

In [62]:
target_df.isnull().sum()

Cluster_Topic                       0
PY                                  0
Topic_Year                          0
PT                                  0
PG                                  0
Have_Funding_Agency                 0
Author_count                        0
Funding_agency_count                0
Squeezed_Publisher                  0
Contain_Top_All_authors             0
Contain_Top_All_funding_agencies    0
Count_of_top_funding_agency         0
Contain_Top_PU                      0
dtype: int64

In [63]:
target_df.shape

(46169, 13)

In [64]:
target_df.head()

,Cluster_Topic,PY,Topic_Year,PT,PG,Have_Funding_Agency,Author_count,Funding_agency_count,Squeezed_Publisher,Contain_Top_All_authors,Contain_Top_All_funding_agencies,Count_of_top_funding_agency,Contain_Top_PU
0,0,2000,0_2000,J,14,0,3,0,ELSEVIER SCIENCE BV,0.0,0.0,0,1.0
1,7,2000,7_2000,J,6,0,3,0,ELSEVIER SCIENCE BV,0.0,0.0,0,1.0
2,11,2000,11_2000,J,9,0,4,0,ELSEVIER SCIENCE BV,0.0,0.0,0,1.0
3,14,2000,14_2000,J,13,0,3,0,Other Publishers,0.0,0.0,0,1.0
4,5,2000,5_2000,J,8,0,6,0,ELSEVIER SCIENCE BV,0.0,0.0,0,1.0


In [ ]:
target_df.to_csv('final_data.csv',index=False)